# Animal Faces

## Задание:

Необходимо реализовать систему классификации лиц животных. Задача представляет из себя классификацию на 3 класса.

1) Необходимо загрузить изображения и привести их к единому формату. Важно понимать, что исходное разрешение очень больше, и простые сети на несколько слоев могут выдать не такое хорошее качество.

2) Необходимо обучить нейросеть для распознавания 3 классов, желательно будет сравнить результаты на нескольких параметрах.

3) Оценить по метрикам качества для каждого класса.

Важно визуализировать результат прогноза (на тесте) в человеко-читаемом формате с подписью.

### Подготовка данных

На этом этапе выполняется подготовка датасета для дальнейшего обучения нейросети.  
Исходные изображения загружаются из Kaggle с помощью kagglehub и сохраняются локально без изменения оригинальной структуры.

Далее выполняется предварительная обработка изображений. Все файлы приводятся к единому формату, цветовой модели RGB и фиксированному размеру. Результат сохраняется в отдельную директорию processed, при этом полностью сохраняется исходная иерархия папок train, val и классов.

Такой подход позволяет не изменять исходные данные, ускоряет дальнейшую загрузку изображений и делает эксперименты воспроизводимыми. Аугментации на этом этапе не применяются и будут добавлены позже при обучении модели.

In [1]:
from scripts.download import download_dataset
from scripts.preprocess import preprocess_dataset
from pathlib import Path


raw_data_dir: Path = download_dataset(dataset_id="andrewmvd/animal-faces", target_dir="data/animal-faces")
processed_data_dir = "data/processed"

processed_path: Path = preprocess_dataset(
    raw_dir=raw_data_dir,
    processed_dir=processed_data_dir,
    image_size=224,
)

### Downloading dataset

Dataset id `andrewmvd/animal-faces`

Target directory `/home/garret/git/mfti/llm_hw5_pytorch/data/animal-faces`

Force download `False`

### Download skipped

...

Target directory `/home/garret/git/mfti/llm_hw5_pytorch/data/processed`

Image size `224 x 224`

Force reprocessing `False`

### Preprocessing skipped

Processed data already exists on disk

### Загрузка подготовленных данных

На этом этапе обработанные изображения подключаются к PyTorch через DataLoader. Используется заранее подготовленная структура данных с разделением на обучающую и валидационную выборки. Загрузка выполняется таким образом, чтобы изображения сразу были приведены к формату, совместимому с предобученной моделью.


In [2]:
from scripts.data import make_dataloaders

processed_path = processed_path / "afhq"
train_loader, val_loader, class_names = make_dataloaders(
    data_dir=processed_path,
    batch_size=32,
    num_workers=2,
)

images, labels = next(iter(train_loader))

images.shape, labels.shape, class_names


### Loading processed dataset

Dataset directory `/home/garret/git/mfti/llm_hw5_pytorch/data/processed/afhq`

Batch size `32`

### Dataset loaded

Number of classes `3`

Class names `['cat', 'dog', 'wild']`

Train samples `14630`

Validation samples `1500`

(torch.Size([32, 3, 224, 224]), torch.Size([32]), ['cat', 'dog', 'wild'])

### Проверка корректности загрузки данных

Был извлечен один батч изображений из обучающей выборки для проверки корректности пайплайна. Форма тензоров подтверждает, что данные успешно загружаются, имеют ожидаемое разрешение и количество каналов. Список классов корректно соответствует структуре датасета. Это означает, что данные готовы к использованию на этапе обучения модели.


### Построение модели

На данном этапе формируется архитектура нейросети, которая будет использоваться для классификации изображений животных. В качестве основы выбрана предобученная сверточная сеть, позволяющая эффективно использовать уже выученные визуальные признаки. Архитектура адаптируется под конкретную задачу за счет изменения выходного слоя и подготовки модели к обучению.


In [5]:
import torch
from scripts.model import build_model

device = "cuda" if torch.cuda.is_available() else "cpu"

model = build_model(
    num_classes=3,
    freeze_backbone=True,
    device=device,
)

### Building model

Architecture `ResNet18`

Number of classes `3`

Freeze backbone `True`

Device `cuda`

### Model ready

Classifier in_features `512`

Classifier out_features `3`

### Инициализация модели

На этом этапе была создана модель классификации изображений на основе предобученной архитектуры ResNet18. Была использована версия модели с весами, обученными на ImageNet, что позволяет сразу использовать уже выученные визуальные признаки. Последний классификационный слой был заменен и адаптирован под задачу с тремя классами. Сверточная часть модели заморожена, обучение на данном этапе будет происходить только для последнего линейного слоя. Это дает стабильный и быстрый базовый результат и позволяет проверить работоспособность всего пайплайна. Модель успешно перенесена на GPU, что подтверждает корректную настройку окружения и готовность к дальнейшему обучению.
